<a href="https://colab.research.google.com/github/sujeetgund/Langchain_rag/blob/main/Langchain_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Necessary Packages

In [ ]:
%pip install langchain langchain_community langchain-core langchain-google-genai langchain-groq pypdf

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Setup Environment Variables

In [2]:
import getpass
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google: ")

## Model to generate LLM response

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="deepseek-r1-distill-llama-70b")

## Model to create vector embeddings

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

## InMemory Vector Store from Langchain_core

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

## Documents and Loader

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4

In [8]:
loader = WebBaseLoader(
    web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_=['post-title', 'post-header', 'post-content'])
    }
)

docs = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents=docs)
all_splits[0]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refin

## Upload Documents in Vector Store

In [11]:
_ = vector_store.add_documents(documents=all_splits)

## Prompt Template

In [12]:
from langchain_core.prompts import PromptTemplate

In [14]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are an AI assistant with access to relevant retrieved documents. Answer the user's query based only on the provided context.
                ### Context (Retrieved Information):
                {context}

                ### User Query:
                {question}

                ### Instructions for Response:
                - Generate a precise and well-structured response using the given context.
                - If the answer is not found in the context, state that the information is unavailable rather than guessing.
                - Maintain clarity, accuracy, and a professional tone.
                - Do not include external knowledge beyond the provided context.

                ### Response:"""
)

## Retreive and Generate Functions

In [16]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

In [17]:
# Define state for rag app
class State(TypedDict):
  question: str
  context: List[Document]
  answer: str

In [18]:
# Function to retrieve similar documents
def retrieve(state: State):
  retrieved_docs = vector_store.similarity_search(state["question"], k=2)
  return {"context": retrieved_docs}

In [19]:
# Function to generate response based on query and context
def generate(state: State):
  docs_content = "\n\n".join([doc.page_content for doc in state["context"]])
  messages = prompt.invoke({"question": state["question"], "context": docs_content})
  response = model.invoke(messages)
  return {"answer": response.content}


In [20]:
app_state: State =  {
  "question": "",
  "context": [],
  "answer": ""
}

## Response from Retrieve Function

In [21]:
app_state["question"] = "What is ReAct?"

retrieved_response = retrieve(app_state)

retrieved_response

{'context': [Document(id='752622d8-56a5-411c-9d29-a2dea4967142', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='ReAct (Yao et al. 2023) integrates reasoning and acting within LLM by extending the action space to be a combination of task-specific discrete actions and the language space. The former enables LLM to interact with the environment (e.g. use Wikipedia search API), while the latter prompting LLM to generate reasoning traces in natural language.\nThe ReAct prompt template incorporates explicit steps for LLM to think, roughly formatted as:\nThought: ...\nAction: ...\nObservation: ...\n... (Repeated many times)'),
  Document(id='0308b713-9c63-4a61-b1ea-147caee84e1f', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='ReAct (Yao et al. 2023) integrates reasoning and acting within LLM by extending the action space to be a combination of task-specific discrete actions and the language space. The forme

## Response from Generate Function

In [22]:
app_state["context"] = retrieved_response["context"]

generated_response = generate(app_state)

generated_response.get("answer")

"<think>\nOkay, so I need to figure out what ReAct is based on the provided context. Let me read through the context again to make sure I understand it correctly.\n\nThe context mentions that ReAct was introduced by Yao et al. in 2023. It integrates reasoning and acting within large language models (LLMs). It does this by extending the action space to include both task-specific discrete actions and language space. The discrete actions allow the LLM to interact with the environment, like using a Wikipedia search API. On the other hand, the language space prompts the LLM to generate reasoning traces in natural language.\n\nThere's also a prompt template described, which includes explicit steps like Thought, Action, Observation, and these steps are repeated many times. This structure helps the LLM to think through a problem step-by-step, perform actions, and process observations from those actions.\n\nSo, putting this together, ReAct is a framework that enhances LLMs by combining their ab